In [1]:
# Install tsflex and seglearn
!pip install tsflex --no-index --find-links=file:///kaggle/input/time-series-tools
!pip install seglearn --no-index --find-links=file:///kaggle/input/time-series-tools

Looking in links: file:///kaggle/input/time-series-tools
Processing /kaggle/input/time-series-tools/tsflex-0.3.0-py3-none-any.whl
Looking in links: file:///kaggle/input/time-series-tools
Processing /kaggle/input/time-series-tools/seglearn-1.2.5-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
from sklearn import *
import glob
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
from pathlib import Path
from seglearn.feature_functions import base_features, emg_features
from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.base import clone
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.utils import resample
from scipy.fft import rfft
import gc
import sys
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold


In [3]:
from numpy.random import seed
seed(1123) 
import random as pyrandom
pyrandom.seed(1123)

In [4]:
# Reduce Memory Usage
# reference : https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN

def reduce_memory_usage(df):
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage became: ",mem_usg," MB")
    
    return df

In [5]:
root = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

train = glob.glob(path.join(root, 'train/**/**'))
# train2 = glob.glob(path.join(root, 'unlabeled/**'))
test = glob.glob(path.join(root, 'test/**/**'))

subjects = pd.read_csv(path.join(root, 'subjects.csv'))
tasks = pd.read_csv(path.join(root, 'tasks.csv'))
events = pd.read_csv(path.join(root, 'events.csv'))

tdcsfog_metadata = pd.read_csv(path.join(root, 'tdcsfog_metadata.csv'))
defog_metadata = pd.read_csv(path.join(root, 'defog_metadata.csv')) 

tdcsfog_metadata['Module'] = 'tdcsfog'
defog_metadata['Module'] = 'defog'

full_metadata = pd.concat([tdcsfog_metadata, defog_metadata])

In [6]:
best_params_ = {'colsample_bytree': 0.5282057895135501,
 'learning_rate': 0.22659963168004743,
 'max_depth': 8,
 'min_child_weight': 3.1233911067827616,
 'n_estimators': 291,
 'subsample': 0.9961057796456088,
#  'objective':'multiclass',
#   'metric':'multi_logloss',
#   'num_class' : 4

#   'device':'gpu'
    
                
 }
# from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def custom_average_precision(y_true, y_pred):
    score = average_precision_score(np.round(y_true), y_pred)
    return 'average_precision', score, True

def custom_average_precision1(y_true, y_pred):
    score = average_precision_score(np.round(y_true), y_pred)
    return score

class LGBMMultiOutputClassifier(MultiOutputClassifier):
    def fit(self, X, y, eval_set=None, **fit_params):
        self.estimators_ = [clone(self.estimator) for _ in range(y.shape[1])]
        
        for i, estimator in enumerate(self.estimators_):
            if eval_set:
                fit_params['eval_set'] = [(eval_set[0], eval_set[1][:, i])]
            estimator.fit(X, y[:, i], **fit_params)
#             calibrated_clf = CalibratedClassifierCV(estimator, cv="prefit")
#             calibrated_clf.fit(eval_set[0], eval_set[1][:, i])
        
        return self
    
    def predict_proba(self, X):
        predictions = np.hstack([estimator.predict_proba(X)[:, 1].reshape(-1, 1) for estimator in self.estimators_])
        return predictions

In [7]:
basic_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(base_features()),
#     series_names=['AccVfft', 'AccMLfft', 'AccAPfft'],
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[1000],
    strides=[800],
)
basic_feats

emg_feats = emg_features()
del emg_feats['simple square integral'] # is same as abs_energy (which is in base_features)
emg_feats

emg_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(emg_feats),
#     series_names=['AccVfft', 'AccMLfft', 'AccAPfft'],
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[1000],
    strides=[800],
)

fc = FeatureCollection([basic_feats, emg_feats])

In [8]:
# trainnt_files = glob.glob(path.join(root, 'train/notype/**')) 

# def reader2(file):
# #     try:        
#         df = pd.read_csv(file)
#         path_split = file.split('/')         
#         dataset = Path(file).parts[-2]  
#         df['Id'] = path_split[-1].split('.')[0]
#         idv=df['Id'][0]       
#         df['Time_frac2']=(df.index/df.index.max()).values
# #         df['Module'] = dataset      

#         df['Flag']= df['Event']        
         
# #         # Reindex based on time
# #         df = df.set_index('Time')
        
#         # Extract specific columns
#         columns_to_select = ['Time_frac2','AccV', 'AccML', 'AccAP', 'Flag']
#         try:
#             df = df[columns_to_select]  

#             df['Module'] = dataset
#             df['Id'] = path_split[-1].split('.')[0]  
            
#             df['Time_frac']=(df.index/df.index.max()).values
# #             df['AccVfft'] = np.abs(rfft(df[['AccV']]))
# #             df['AccMLfft'] = np.abs(rfft(df[['AccML']]))
# #             df['AccAPfft'] = np.abs(rfft(df[['AccAP']]))
            

#             df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
#             df = df.merge(df_feats, how="left", left_index=True, right_index=True)   

#             df.fillna(method="ffill", inplace=True)

#             return df
#         except: 
#             print("error at df, ignored",idv)
#             pass
# #         print("file name exception ",file)
# #           pass

# trainnt = pd.concat([reader2(f) for f in tqdm(trainnt_files)]).fillna(0)

# # cols = [c for c in train.columns if c not in ['Id','Subject','Module', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]
# # pcols = ['StartHesitation', 'Turn' , 'Walking']
# # scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']
# trainnt=trainnt.reset_index(drop=True)

# trainnt["total_accl"]= np.sqrt(np.square(trainnt.AccV) + np.square(trainnt.AccML) + np.square(trainnt.AccAP))
# trainnt["abs_delta1"] = (trainnt["total_accl"] - trainnt["total_accl"].shift(100)) * 1000
# trainnt["abs_delta2"] = (trainnt["total_accl"] - trainnt["total_accl"].shift(1000)) * 1000
# trainnt["abs_delta1"] = trainnt["abs_delta1"].fillna(0)
# trainnt["abs_delta2"] = trainnt["abs_delta2"].fillna(0)

# # print("columns in train data ",len(train.columns))
# print("columns in trainnt data ",len(trainnt.columns))
# # trainnt.columns

In [9]:
def endtransform(df):
    
    df=pd.DataFrame(df)
    ones_indices = np.where(df == 1)[0]  # Get indices of ones
     
    indx={}
    k=0
    diff=0
    nex=0
    ind={}
    j=0
    for i in range(len(ones_indices)):
        indx['start']=ones_indices[k]         
        prev = ones_indices[i]        
        nex = ones_indices[i+1] if i < len(ones_indices) - 1 else len(df)         
        diff = nex -prev        

        if diff> 1:  
            indx['end']=prev
            ind[j]=(indx['start'],indx['end'])
            j+=1             
            k= i+1 

    for key in ind.keys():
        k=2
        p=0.2
        start=ind[key][0]
        end = ind[key][1] 
        dur=end-start
        df[start:start+round(dur*p)]=df[start:start+round(dur*p)]*k
        df[end+1 - round(dur*p):end+1]=df[end+1 - round(dur*p):end+1]*k
    return df


In [10]:
# from sklearn.metrics import confusion_matrix
 
# from sklearn.model_selection import train_test_split

# # model1 = lgb.LGBMClassifier() 

# best_params_ = {'colsample_bytree': 0.5282057895135501,
#  'learning_rate': 0.22659963168004743,
#  'max_depth': 8,
#  'min_child_weight': 3.1233911067827616,
#  'n_estimators': 291,
#  'subsample': 0.9961057796456088,
 

# #   'device':'gpu'
    
                
#  }
# model2 = lgb.LGBMRegressor(**best_params_)  

# # train['Flag'] = endtransform(train['Flag'])
# # trainnt['Flag']= endtransform(trainnt['Flag'])

# # train3=train.drop(['Time_frac','t_group','Visit','Test','Medication','Id','Subject','Module', 
# #                    's_group','label'],axis=1).sample(n=1000000, random_state=1123)
# trainnt=trainnt.drop(['Module','Id'],axis=1).sample(n=2000000,random_state=1123) #,weights='Flag')
# # train4 = pd.concat([train3,trainnt])
# # trainnt=trainnt.sample(n=200000,random_state=1123)

# y = trainnt['Flag']
# X = trainnt.drop(['Flag'],axis=1)

# # print(y[y.Flag])

# ntcols = X.columns.values
# print("nt cols ",ntcols)

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1123)

# # # Perform random undersampling on the training data
# # rus = RandomUnderSampler(random_state=1123)
# # X_train_res, y_train_res = rus.fit_resample(X_train, y_train) 

# y_train  = endtransform(y_train) 

# # Train the model on the undersampled training data
# # model1.fit(X_train_res, y_train_res)
# model2.fit(X_train, y_train)

# print("shape of X_train ",X_train.shape)

# # Predict on the test data
# # y_pred = model1.predict(X_test)
# y_pred2 = model2.predict(X_test) 

# print('mse is ',mean_squared_error(y_test,y_pred2))

# # confusion_matrix = confusion_matrix(y_test,y_pred2)
# # print("confusion matrix \n",confusion_matrix)
# # classification_report = classification_report(y_test,y_pred2)

# # print("classification report \n",classification_report)

In [11]:
# import sys
# sys.getsizeof(trainnt)

In [12]:
# del trainnt
# gc.collect()

In [13]:
subjects.loc[subjects['Subject'] == 'fe5d84', 'Sex'] = 'F' 

seed = 1123
cluster_size = 10

subjects['Sex'] = subjects['Sex'].factorize()[0]
subjects = subjects.fillna(0).groupby('Subject').median()
subjects['s_group'] = cluster.KMeans(n_clusters = cluster_size, random_state = seed).fit_predict(subjects[subjects.columns[1:]])
new_names = {'Visit':'s_visit','Age':'s_age','YearsSinceDx':'s_years','UPDRSIII_On':'s_on','UPDRSIII_Off':'s_off','NFOGQ':'s_NFOGQ', 'Sex': 's_sex'}
subjects = subjects.rename(columns = new_names)
subjects

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,s_visit,s_age,s_sex,s_years,s_on,s_off,s_NFOGQ,s_group
Subject,,,,,,,,
00f674,1.5,63.0,0.0,27.0,37.0,39.5,25.0,1
02bc69,0.0,69.0,0.0,4.0,21.0,0.0,22.0,0
040587,1.5,75.0,0.0,26.0,49.5,72.0,22.5,7
056372,2.0,69.0,0.0,13.0,44.0,50.0,22.0,4
07285e,0.0,58.0,0.0,1.0,18.0,26.0,10.0,6
...,...,...,...,...,...,...,...,...
f686f0,0.0,61.0,0.0,7.0,44.0,0.0,24.0,8
f80507,1.0,57.0,0.0,2.0,12.0,0.0,0.0,5
fa8764,0.0,60.0,1.0,7.0,30.0,0.0,19.0,0


In [14]:
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks['t_group'] = cluster.KMeans(n_clusters = cluster_size, random_state = seed).fit_predict(tasks[tasks.columns[1:]])

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [15]:
# merge the subjects with the metadata
metadata_w_subjects = full_metadata.merge(subjects, how='left', on='Subject').copy()
features = metadata_w_subjects.columns
metadata_w_subjects['Medication'] = metadata_w_subjects['Medication'].factorize()[0]
metadata_w_subjects

,Id,Subject,Visit,Test,Medication,Module,s_visit,s_age,s_sex,s_years,s_on,s_off,s_NFOGQ,s_group
0,003f117e14,4dc2f8,3,2.0,0,tdcsfog,0.0,68.0,1.0,9.0,17.0,15.0,15.0,6
1,009ee11563,f62eec,4,2.0,0,tdcsfog,0.0,71.0,0.0,10.0,42.0,0.0,24.0,8
2,011322847a,231c3b,2,2.0,0,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,2
3,01d0fe7266,231c3b,2,1.0,1,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,2
4,024418ba39,fa8764,19,3.0,0,tdcsfog,0.0,60.0,1.0,7.0,30.0,0.0,19.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,1a778d,1,NaN,1,defog,1.5,65.0,0.0,7.0,50.0,59.5,24.5,7
966,f40e8c6ebe,575c60,1,NaN,1,defog,1.0,28.0,0.0,4.0,54.0,50.0,25.0,1
967,f8ddbdd98d,107712,1,NaN,0,defog,1.0,82.0,1.0,11.0,38.0,42.0,21.0,4
968,f9efef91fb,5d9cae,2,NaN,1,defog,1.5,72.0,0.5,14.0,22.5,39.0,16.0,2


In [16]:
import scipy as sp
from scipy import signal
from scipy import fftpack
import pywt

def detect_steps(signal, frequency,smoothing=2, faster_please=True, min_scale_log=3.6, max_scale_log=5.4):
    
    # scales for wavelets that define frequencies, exp for more even distribution
    # choosing border values is tricky
    # too narrow and we lose some slow or fast steprates
    # too wide and we get the steprate wrong for unusual walking wavelet spectrum cases
    # check patient #6 in the tdcsfog dataset as an example of an unusual pattern
    scales = np.exp(np.arange(min_scale_log, max_scale_log, 0.05))
    wavelet='morl'  # chosing the Morlet wavelet
    
    # transforming the signal, preferrably the 'AccMl' one
    coeff, freq = pywt.cwt(signal, scales, wavelet)
    # finding the brightest dots on every time slice
    coeff_argmax_index = np.argmax(abs(coeff), 0)
    # and collecting their coeff values in a new "signal", which already looks like our step rate, but is a bit noisy
    coeff_max = np.array([coeff[coeff_argmax_index[i], i] for i in range(coeff.shape[1])])
    
    # repeating the smoothing procedure
    for i in range(smoothing):
        # transforming the noisy step rate again to clear it
        coeff, freq = pywt.cwt(coeff_max, scales, wavelet)
        # finding the brightest dots on every time slice
        coeff_argmax_index = np.argmax(abs(coeff), 0).astype(int)
        # and smoothing their indices with median to cut some outbursts off
        coeff_argmax_index = np.round(pd.Series(coeff_argmax_index).rolling(128, center=True, min_periods=1).median()).astype(int)
        # collecting the values along our line of indices
        coeff_max = np.array([coeff[coeff_argmax_index[i], i] for i in range(coeff.shape[1])])
    
    # this smoothing round is optional
    if not faster_please:
        # finding hopefully bigger peaks on our line
        # hopefully skiping too narrow or too close ones that are likely to be outbursts
        max_cwt_points = sp.signal.find_peaks(abs(coeff_max), distance=20, width=20)[0]
        max_cwt_points = np.concatenate(([0], max_cwt_points, [len(signal)-1]))
        # and interpolating our line in wavelet space between the bigger peaks
        max_cwt_line_indexes = np.round(np.interp( range(0, len(signal)), max_cwt_points, 
                                                    coeff_argmax_index[max_cwt_points])).astype(int)
        coeff_max = np.array([coeff[max_cwt_line_indexes[i], i] for i in range(coeff.shape[1])])
    
    # finding zeroes on this smooth line to separate individual steps from other steps    
    zero_crossings = np.where( np.diff(np.sign(pd.Series(coeff_max).rolling(10, center=True, min_periods=1).mean())))[0]
    zero_crossings = np.concatenate( ([0], zero_crossings, [len(signal)]))    
    #filling each step with its duration while cutting possible outbursts with median
    step_lengths = []
    for i in range(1, len(zero_crossings)):
        step_lengths = np.concatenate( (step_lengths, [zero_crossings[i] - zero_crossings[i-1]]*(zero_crossings[i] - zero_crossings[i-1]) ))
    step_durations = pd.Series(step_lengths).rolling(32, center=True, min_periods=1).median()
    # making a nice and smooth step_rate array
    
    step_rate = pd.Series( 1./step_durations )*frequency
    step_rate = step_rate.where(step_rate<5, 0).rolling(frequency, center=True, min_periods=1).mean()
    
    return step_rate, step_durations 






In [17]:
def reader(file):
#     try:        
        df = pd.read_csv(file)
        path_split = file.split('/')         
        dataset = Path(file).parts[-2]  
        df['Id'] = path_split[-1].split('.')[0]
        idv=df['Id'][0]       
        df['Time_frac2']=(df.index/df.index.max()).values
        df['annot']=1
        if dataset!='notype':
            df['Flag']= np.any(df[['StartHesitation','Turn','Walking']] == 1, axis=1).astype(int)
        
#         print("in df ",idv)
        
        if events[events.Id==idv].empty==False:
            
#             ms=(events.Id==idv) & (events.Type=='StartHesitation')
#             mt=(events.Id==idv) & (events.Type=='Turn')
#             mw=(events.Id==idv) & (events.Type=='Walking')
            
            if dataset=='defog':
                df['annot']=0
                df.loc[df[(df.Valid==True) & (df.Task==True)].index.values,'annot']=1
                start =  round(events[events.Id==idv].Init.min() * 100)
                end = round(events[events.Id==idv].Completion.max() * 100)
                
            else:
                start =  round(events[events.Id==idv].Init.min() * 128)
                end = round(events[events.Id==idv].Completion.max() * 128)
                
            dur = end - start
#             print("dur is ",dur)
            start = round(max(start - 2 * dur,0))
            end   = round(end + 2 * dur)               
            if end >= df.Time.max(): end=df.Time.max()                
            indx = list(np.arange(start,end)) 
            df = df.loc[indx] 
        try:
            df['None'] = (df['StartHesitation'] == 0) & (df['Turn'] == 0) & (df['Walking'] == 0)
    # Convert the boolean values to integers (0 and 1)
            df['None'] = df['None'].astype(int)    

#             for index, row in df.iterrows():
#                 for col in ['None','StartHesitation','Turn','Walking']:
#                     if row[col] == 1:
#                         df.at[index, 'label'] = col
#                         break

#             df['label'] = df['label'].astype(str)

        # # Perform label encoding on the 'Label' column

    #             except:
    #                 pass

    # # # Print the resulting DataFrame


    #             ev = events[events.Id==idv] 

    #             print("in df ",idv)

    #             if dataset!='notype':
    #                 for i,row in ev.iterrows():

    #                     if dataset=="defog":
    #                         start_r = round(row.Init * 100)
    #                         end_r = round(row.Completion * 100)

    #                     else:
    #                         start_r = round(row.Init * 128)
    #                         end_r = round(row.Init * 128) 

    #                     durr = end_r - start_r

    #                     if start_r!=0:               
    #     #                     print("dafdsf \n",df.loc[start_r:(start_r+round(0.1*durr)),:])
    #                         df.loc[start_r:(start_r+round(0.2*durr)),'StartHesitation'] = df.loc[start_r:(start_r+round(0.2*durr)),'StartHesitation'] * 0.8
    #                         df.loc[(end_r - round(0.2*durr)):end_r,'StartHesitation'] = df.loc[(end_r - round(0.2*durr)):end_r,'StartHesitation'] * 0.8

    #                         df.loc[start_r:(start_r+round(0.2*durr)),'Turn'] = df.loc[start_r:(start_r+round(0.2*durr)),'Turn'] * 0.8
    #                         df.loc[(end_r - round(0.2*durr)):end_r,'Turn'] = df.loc[(end_r - round(0.2*durr)):end_r,'Turn'] * 0.8

    #                         df.loc[start_r:(start_r+round(0.2*durr)),'Walking'] = df.loc[start_r:(start_r+round(0.2*durr)),'Walking'] * 0.8
    #                         df.loc[(end_r - round(0.2*durr)):end_r,'Walking'] = df.loc[(end_r - round(0.2*durr)):end_r,'Walking'] * 0.8

            # Reindex based on time
            df = df.set_index('Time')

            # Extract specific columns
            columns_to_select = ['Time_frac2','annot','None','AccV', 'AccML', 'AccAP', 'StartHesitation','Turn','Walking','Flag'] #'annot',
    #         try:
            df = df[columns_to_select]  

            df['Module'] = dataset
            df['Id'] = path_split[-1].split('.')[0]

    #             # this is done because the speeds are at different rates for the datasets
    #             if dataset == 'tdcsfog':
    #                 df.AccV = df.AccV / 9.80665
    #                 df.AccML = df.AccML / 9.80665
    #                 df.AccAP = df.AccAP / 9.80665

            df['Time_frac']=(df.index/df.index.max()).values
#             df['AccVfft'] = np.abs(rfft(df[['AccV']]))
#             df['AccMLfft'] = np.abs(rfft(df[['AccML']]))
#             df['AccAPfft'] = np.abs(rfft(df[['AccAP']]))



            df = pd.merge(df, tasks[['Id','t_group']], how='left', on='Id').fillna(-1)


            df = pd.merge(df, metadata_w_subjects[['Id','Subject', 'Visit','Test','Medication','s_group']], how='left', on='Id').fillna(-1)

            df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
            df = df.merge(df_feats, how="left", left_index=True, right_index=True) 
            
            if dataset=='defog':
                df['step_rate'], df['step_durations']= detect_steps(df['AccML'],frequency=100)
            if dataset == 'tdcsfog':
                df['step_rate'], df['step_durations'] = detect_steps(df['AccML'],frequency=128)


            df.fillna(method="ffill", inplace=True)

        #             print(" df completed ",idv)

            return df
        except: 
            print("error at df, ignored",idv)
            pass
#         print("file name exception ",file)
#           pass

train = pd.concat([reader(f) for f in tqdm(train)]).fillna(0); print(train.shape)

cols = [c for c in train.columns if c not in ['Id','Subject','Module', 'Time', 'None','StartHesitation','Turn','Walking','Valid', 'Task','Event']]
pcols = ['None','StartHesitation', 'Turn' , 'Walking']
scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']
train=train.reset_index(drop=True)

  0%|          | 0/970 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/seglearn/feature_functions.py:185: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return stats.skew(X, axis=1)
/opt/conda/lib/python3.10/site-packages/seglearn/feature_functions.py:190: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return stats.kurtosis(X, axis=1)
/opt/conda/lib/python3.10/site-packages/seglearn/feature_functions.py:190: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return stats.kurtosis(X, axis=1)
/opt/conda/lib/python3.10/site-packages/seglearn/feature_functions.py:185: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic canc

error at df, ignored 1e8d55d48d
error at df, ignored 89e9ed32d1
error at df, ignored e5a0e226fe
error at df, ignored 1b3bc93401
error at df, ignored 34b979fc28
error at df, ignored 9cd837fd53
error at df, ignored 60f28aa837
error at df, ignored 02ab235146
error at df, ignored 6a886a3bb8
error at df, ignored 339c0cc15f
error at df, ignored af02b83cbf
error at df, ignored 71dd8ce20d
error at df, ignored b1f5aa1b77
error at df, ignored 296c84448e
error at df, ignored 46cdfe23ea
error at df, ignored 2cc3c30645
error at df, ignored 8228c6fdee
error at df, ignored 847ebc0a8d
error at df, ignored 2acdf5a450
error at df, ignored dd589529b5
error at df, ignored ad8e83242a
error at df, ignored d2ecd51a66
error at df, ignored 9506859311
error at df, ignored 0a900ed8a2
error at df, ignored affdf8553f
error at df, ignored 72853af746
error at df, ignored cb82000108
error at df, ignored 6214414fff
error at df, ignored 2054f1d5df
error at df, ignored 6614bf9767
error at df, ignored 6a18e3751f
error at

In [18]:
train["total_accl"]= np.sqrt(np.square(train.AccV) + np.square(train.AccML) + np.square(train.AccAP))
train["abs_delta1"] = (train["total_accl"] - train["total_accl"].shift(100)) * 1000
train["abs_delta2"] = (train["total_accl"] - train["total_accl"].shift(1000)) * 1000
train["abs_delta1"] = train["abs_delta1"].fillna(0)
train["abs_delta2"] = train["abs_delta2"].fillna(0)

In [19]:
train

,Time_frac2,annot,None,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Flag,...,AccV__std__w=1000,AccV__var__w=1000,AccV__waveform_length__w=1000,AccV__willison_amplitude__w=1000,AccV__zero_crossing__w=1000,step_rate,step_durations,total_accl,abs_delta1,abs_delta2
0,0.256560,0,1,-0.990583,0.034405,0.220923,0,0,0,0,...,0.033435,0.001118,6.861675,999.0,999.0,1.785714,56.0,1.015503,0.000000,0.000000
1,0.256568,0,1,-0.989055,0.029341,0.217371,0,0,0,0,...,0.033435,0.001118,6.861675,999.0,999.0,1.785714,56.0,1.013085,0.000000,0.000000
2,0.256577,0,1,-0.987427,0.026044,0.217463,0,0,0,0,...,0.033435,0.001118,6.861675,999.0,999.0,1.785714,56.0,1.011425,0.000000,0.000000
3,0.256585,0,1,-0.986689,0.024334,0.219943,0,0,0,0,...,0.033435,0.001118,6.861675,999.0,999.0,1.785714,56.0,1.011199,0.000000,0.000000
4,0.256593,0,1,-0.985536,0.023370,0.220194,0,0,0,0,...,0.033435,0.001118,6.861675,999.0,999.0,1.785714,56.0,1.010105,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17346445,0.821214,1,1,-9.366973,-1.455658,3.568429,0,0,0,0,...,2.115103,4.473661,128.436508,239.0,239.0,2.061791,61.0,10.128811,1932.269462,91.936100
17346446,0.821408,1,1,-10.462647,-0.939006,3.965941,0,0,0,0,...,2.115103,4.473661,128.436508,239.0,239.0,2.065599,61.0,11.228420,3509.212101,1291.277977
17346447,0.821602,1,1,-11.639707,-0.373443,4.108541,0,0,0,0,...,2.115103,4.473661,128.436508,239.0,239.0,2.069521,61.0,12.349184,5035.685261,2507.438248
17346448,0.821796,1,1,-12.478907,0.355374,4.171699,0,0,0,0,...,2.115103,4.473661,128.436508,239.0,239.0,2.073562,61.0,13.162541,5849.630230,3384.464924


In [20]:
# train['Flag']= np.any(train[['StartHesitation','Turn','Walking']] == 1, axis=1).astype(int)
cols = cols + ['total_accl','abs_delta1','abs_delta2'] 

In [21]:
reduce_memory_usage(train)

Memory usage of dataframe is 6881.83 MB
Memory usage became:  2564.1874561309814  MB


,Time_frac2,annot,None,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Flag,...,AccV__std__w=1000,AccV__var__w=1000,AccV__waveform_length__w=1000,AccV__willison_amplitude__w=1000,AccV__zero_crossing__w=1000,step_rate,step_durations,total_accl,abs_delta1,abs_delta2
0,0.256592,0,1,-0.990723,0.034393,0.220947,0,0,0,0,...,0.033447,0.001118,6.863281,999.0,999.0,1.786133,56.0,1.015625,0.0,0.0000
1,0.256592,0,1,-0.989258,0.029343,0.217407,0,0,0,0,...,0.033447,0.001118,6.863281,999.0,999.0,1.786133,56.0,1.012695,0.0,0.0000
2,0.256592,0,1,-0.987305,0.026047,0.217407,0,0,0,0,...,0.033447,0.001118,6.863281,999.0,999.0,1.786133,56.0,1.011719,0.0,0.0000
3,0.256592,0,1,-0.986816,0.024338,0.219971,0,0,0,0,...,0.033447,0.001118,6.863281,999.0,999.0,1.786133,56.0,1.010742,0.0,0.0000
4,0.256592,0,1,-0.985352,0.023376,0.220215,0,0,0,0,...,0.033447,0.001118,6.863281,999.0,999.0,1.786133,56.0,1.009766,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17346445,0.821289,1,1,-9.367188,-1.456055,3.568359,0,0,0,0,...,2.115234,4.472656,128.375000,239.0,239.0,2.062500,61.0,10.125000,1932.0,91.9375
17346446,0.821289,1,1,-10.460938,-0.938965,3.966797,0,0,0,0,...,2.115234,4.472656,128.375000,239.0,239.0,2.066406,61.0,11.226562,3510.0,1291.0000
17346447,0.821777,1,1,-11.640625,-0.373535,4.109375,0,0,0,0,...,2.115234,4.472656,128.375000,239.0,239.0,2.070312,61.0,12.351562,5036.0,2508.0000
17346448,0.821777,1,1,-12.476562,0.355469,4.171875,0,0,0,0,...,2.115234,4.472656,128.375000,239.0,239.0,2.074219,61.0,13.164062,5848.0,3384.0000


In [22]:
best_params1_ = {
            'tree_method': 'hist',
            'objective': 'multi:softprob',
            'eval_metric': 'map',
            'max_depth': 10, #scope.int(hp.quniform('max_depth', 4, 10, 1)),
            'subsample': 1, #hp.uniform('subsample', 0.6, 0.9),
            'learning_rate': 0.2,
            'min_child_weight': 4, 
            'num_class' : 4
    #hp.quniform('min_child_weight', 1, 20, 1),
#             'reg_alpha': hp.quniform('reg_alpha', 0, 1, .001),
#             'reg_lambda': hp.quniform('reg_lambda', 0, 100, 1),
#             'gamma': hp.uniform ('gamma', 1, 9),
#             'verbosity': 1,
#             'random_state': RANDOM_SEED
        }   

In [23]:
cols.remove('Flag')

In [24]:
# train1 = train[train.Module == 'defog'].reset_index(drop=True)
# train2 = train[train.Module == 'tdcsfog'].reset_index(drop=True)

# del train
# gc.collect()
# train1.drop(['Module'],inplace=True,axis=1)

kfold = GroupKFold(5)
groups=kfold.split(train, groups=train.Subject)

classify = []
cvs = []
# cvs2 = []
# pcols =  ['StartHesitation', 'Turn' , 'Walking']
# pcols = ['label']
for _, (tr_idx, te_idx) in enumerate(tqdm(groups, total=5, desc="Folds")): 
    
    tr_idx = pd.Series(tr_idx).sample(n=2000000,random_state=1123).values 
    
    classifier = lgb.LGBMClassifier(**best_params_)
    ovrclassifier = OneVsRestClassifier(classifier)
    
    cv = StratifiedKFold(n_splits=3)
    calibrated_clf = CalibratedClassifierCV(ovrclassifier, cv=cv)
    
    
    
#     multioutput_classifier = LGBMMultiOutputClassifier(lgb.LGBMClassifier(**best_params_))

    x_train = train.loc[tr_idx, cols]
#     x_train1 =x_train[ntcols]   
#     x_train['Flag'] = pd.Series(model2.predict(x_train1))
#     x_train['Flag'] = endtransform(x_train['Flag'])
    x_train = x_train.to_numpy()
    
 
    y_train = train.loc[tr_idx, pcols].to_numpy()     
     
    x_test = train.loc[te_idx, cols]
#     x_test1 = x_test[ntcols]
#     x_test['Flag']=pd.Series(model2.predict(x_test1))
#     x_test['Flag'] = endtransform(x_test['Flag'])
    x_test = x_test.to_numpy()
    
    y_test = train.loc[te_idx, pcols].to_numpy() 
    
    multioutput_classifier = LGBMMultiOutputClassifier(calibrated_clf).fit(x_train, y_train)

#     multioutput_classifier.fit(
#         x_train, y_train,
#         eval_set=(x_test, y_test),
#         eval_metric=custom_average_precision,
#         early_stopping_rounds=25,
#         verbose = 0,
#     )
    
    classify.append(multioutput_classifier)  
    
#     cv = custom_average_precision1(y_test, multioutput_classifier.predict_proba(x_test)) #.clip(0.0,1.0))
    
#     cvs.append(cv)
# #     cvs2.append(cv2)
    
# print(cvs)
# # print(cvs2)
# print(np.mean(cvs))
# # print(np.mean(cvs2))

Folds:   0%|          | 0/5 [00:00<?, ?it/s]

In [25]:
sub = pd.read_csv(path.join(root, 'sample_submission.csv'))
submission = []

# pcols =  ['StartHesitation', 'Turn' , 'Walking']

for f in test:
    df = pd.read_csv(f)
    df['Time_frac2']=(df.index/df.index.max()).values
    df.set_index('Time', drop=True, inplace=True)

    df['Id'] = f.split('/')[-1].split('.')[0]
    df['annot']=1

    dataset = Path(f).parts[-2]
    
     
            
    df['Time_frac']=(df.index/df.index.max()).values
    
#     df['AccVfft'] = np.abs(rfft(df[['AccV']]))
#     df['AccMLfft'] = np.abs(rfft(df[['AccML']]))
#     df['AccAPfft'] = np.abs(rfft(df[['AccAP']]))
    
    df = pd.merge(df, tasks[['Id','t_group']], how='left', on='Id').fillna(-1)

    df = pd.merge(df, metadata_w_subjects[['Id','Subject', 'Visit','Test','Medication','s_group']], how='left', on='Id').fillna(-1)
    df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin")
    df = df.merge(df_feats, how="left", left_index=True, right_index=True)
    if dataset=='defog':
        df['step_rate'], df['step_durations']= detect_steps(df['AccML'],frequency=100)
    if dataset == 'tdcsfog':
        df['step_rate'], df['step_durations'] = detect_steps(df['AccML'],frequency=128)
    
    
    df["total_accl"]= np.sqrt(np.square(df.AccV) + np.square(df.AccML) + np.square(df.AccAP))
    df["abs_delta1"] = (df["total_accl"] - df["total_accl"].shift(100)) * 1000
    df["abs_delta2"] = (df["total_accl"] - df["total_accl"].shift(1000)) * 1000
    df["abs_delta1"].fillna(0)
    df["abs_delta2"].fillna(0)
    df.fillna(method="ffill", inplace=True)
#     df_classify=df[ntcols]
#     yp=model2.predict(df[ntcols]) 
    
    
#     df['Flag']=pd.Series(yp)
#     df['Flag'] = endtransform(df['Flag'])
    df.fillna(0)     
        
    res_vals = []
    
    for i_fold in range(5):
        pred = classify[i_fold].predict_proba(df[cols]) 
        res_vals.append(np.expand_dims(np.round(pred, 3), axis = 2))            
          
    res_vals = np.mean(np.concatenate(res_vals, axis = 2), axis = 2)
    res = pd.DataFrame(res_vals, columns=pcols)
    
    df = pd.concat([df,res], axis=1)
    
#     df.loc[df['None'] > 0.7, ['StartHesitation', 'Turn', 'Walking']] = 0
    
#     df.drop(['None'],inplace=True, axis=1)
    
    df['Id'] = df['Id'].astype(str) + '_' + df.index.astype(str)
    submission.append(df[scols])
    
submission = pd.concat(submission)


submission = pd.merge(sub[['Id']], submission, how='left', on='Id').fillna(0.0)
os.chdir(r'/kaggle/working/')
submission[scols].to_csv('submission.csv', index=False)
submission

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0.0,0.004,0.0
1,003f117e14_1,0.0,0.004,0.0
2,003f117e14_2,0.0,0.004,0.0
3,003f117e14_3,0.0,0.004,0.0
4,003f117e14_4,0.0,0.004,0.0
...,...,...,...,...
286365,02ab235146_281683,0.0,0.004,0.0
286366,02ab235146_281684,0.0,0.004,0.0
286367,02ab235146_281685,0.0,0.004,0.0
286368,02ab235146_281686,0.0,0.004,0.0


In [26]:
submission['StartHesitation'].sum(),submission['Turn'].sum(),submission['Walking'].sum()
# (3037.7040000000006, 13867.645600000002, 5489.886600000002)
# (2666.168200000001, 9083.0386, 2668.4824)

(0.0028000000000000004, 3608.4236, 753.138)

In [27]:
# sub = pd.read_csv(path.join(root, 'sample_submission.csv'))
# submission = []

# # pcols =  ['None','StartHesitation', 'Turn' , 'Walking']
# pcols =  ['StartHesitation', 'Turn' , 'Walking']

# for f in test:
#     df = pd.read_csv(f)
#     df['Time_frac2']=(df.index/df.index.max()).values
#     df.set_index('Time', drop=True, inplace=True)

#     df['Id'] = f.split('/')[-1].split('.')[0]

#     dataset = Path(f).parts[-2]
        
# #     if dataset == 'tdcsfog':
# #         df.AccV = df.AccV / 9.80665
# #         df.AccML = df.AccML / 9.80665
# #         df.AccAP = df.AccAP / 9.80665
            
#     df['Time_frac']=(df.index/df.index.max()).values
#     df = pd.merge(df, tasks[['Id','t_group']], how='left', on='Id').fillna(-1)

#     df = pd.merge(df, metadata_w_subjects[['Id','Subject', 'Visit','Test','Medication','s_group']], how='left', on='Id').fillna(-1)
#     df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin")
#     df = df.merge(df_feats, how="left", left_index=True, right_index=True)
    
    
#     df["total_accl"]= np.sqrt(np.square(df.AccV) + np.square(df.AccML) + np.square(df.AccAP))
#     df["abs_delta1"] = (df["total_accl"] - df["total_accl"].shift(100)) * 1000
#     df["abs_delta2"] = (df["total_accl"] - df["total_accl"].shift(1000)) * 1000
#     df["abs_delta1"].fillna(0)
#     df["abs_delta2"].fillna(0)
#     df.fillna(method="ffill", inplace=True)
     
#     yp=model2.predict(ntcols)
#     df['Flag']=pd.Series(yp)
# #     df['Flag'] = endtransform(df['Flag'])
#     df.fillna(0)    
        
#     res_vals = []
    
#     for i_fold in range(5):
        
#         pred = regs[i_fold].predict_proba(df[cols])[:,1:] #.clip(0.0,1.0)
# #         print(pred)
# #         dfsd
#         res_vals.append(np.expand_dims(np.round(pred, 3), axis = 2))
        
        
#     res_vals = np.mean(np.concatenate(res_vals, axis = 2), axis = 2)
#     res = pd.DataFrame(res_vals, columns=pcols)
    
#     df = pd.concat([df,res], axis=1)
#     df['Id'] = df['Id'].astype(str) + '_' + df.index.astype(str)
# #     scols1 = ['StartHesitation', 'Turn' , 'Walking','None','Id']
#     submission.append(df[scols])
    
# submission = pd.concat(submission)

# # max_column = submission[['StartHesitation','Turn','Walking','None']].idxmax(axis=1)

# # # Set values of other columns to 0
# # submission.loc[max_column != 'StartHesitation', 'StartHesitation'] = 0
# # submission.loc[max_column != 'Turn', 'Turn'] = 0
# # submission.loc[max_column != 'Walking', 'Walking'] = 0
# # submission.loc[max_column != 'None', 'None'] = 0

# # # Threshold value
# # threshold = 0.5

# # submission['StartHesitation'][submission['StartHesitation'] >= threshold] = 1
# # submission['Turn'][submission['Turn'] >= threshold] = 1
# # submission['Walking'][submission['Walking'] >= threshold] = 1
# # submission['None'][submission['None'] >= threshold] = 1

# # scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']


# submission = pd.merge(sub[['Id']], submission, how='left', on='Id').fillna(0.0)
# os.chdir(r'/kaggle/working/')
# submission[scols].to_csv('submission.csv', index=False)
# # submission

In [28]:
# [0.3651458691967257, 0.20678542041783934, 0.06170097111594116, 0.15119806424902058, 0.06150458186906638]
# 0.16926698136971866

# [0.1262944734525422, 0.13564939054645525, 0.031860946725975, 0.08222518609558244, 0.03200343657865203]
# 0.08160668667984138

# [0.13026471957413796, 0.05112257329301665, 0.027278221087166526, 0.09745428203630054, 0.026953597619237216]
# 0.06661467872197177